<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Варахобова Анна Андреевна, @h.varakhobava.

# <center>Обзор библиотеки статистического моделирования Statsmodels

В этом тьюториале будет рассмотрена библиотека statsmodels и некоторые популярные ее применения.

<a href="https://www.statsmodels.org">statsmodels</a> - это библиотека для статистического моделирования, содержащая всевозможные статистические модели, множество статистических тестов, средств для информативных графиков и др. Распространяется под лицензией New BSD (modified BSD).

Этот пакет покажется знакомым тем, кто работал со статистическим моделированием на R, т.к. позволяет пользоватся строковыми формулами для описания линейной модели, и содержит похожие методы и классы, а также позволяет пользоваться наборами данных из R.


### Линейная регрессия

Посмотрим, как решить задачу линейной регрессии с помощью statsmodels. В пакете statsmodels.api.datasets представлено некоторое количество датасетов, включая датасеты, доступные в R. Воспользуемся одним из них. 

Во встроенных данных не из R присутствуют свойства endog и exog - целевой и все остальные признаки соответственно, а сам массив лежит в свойстве data. В даных из R endog и exog свойств нет, а в data лежит pandas DataFrame.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

%matplotlib inline

import warnings

import seaborn as sns

warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
data = sm.datasets.get_rdataset("Guerry", "HistData").data

In [ ]:
data[:10]

In [ ]:
data['Lottery'].head()

Для обычной линейной регрессии используем OLS - ordinary least squares модель.

Сначала подготовим данные: удалим пропуски и закодируем категориальный признак Region, а затем обучим модель и выведем саммари.

In [ ]:
df = data[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()

df_dumm = pd.get_dummies(df, columns=['Region'], drop_first=True)

# обучим, добавив к предикторам смещение
ols = sm.OLS(df_dumm['Lottery'], sm.add_constant(df_dumm.drop(['Lottery'], axis=1)))
res = ols.fit()

res.summary()

В саммари можно сразу без лишних телодвижений посмотреть статистические характеристики модели: коэффициенты модели, значимы ли они, их доверительные интервалы. Также выводится множество других параметров: метрики качества, значения статистических тестов.

Видим, что модель имеет R2 метрику, равную 0.3, так себе модель :) Высокие p-value коэффициентов говорят о том же.

#### Формулы
Проделаем то же самое, но с использованием формул, а также посмотрим, какие "чудеса" можно вытворять с их помощью.

Говорим модели, что хотим предсказать Lottery как линейную комбинацию остальных перечисленных параметров
добавляем вконце -1, чтобы убрать смещение, т.е. чтобы y = a1x1 + .. + anxn


In [ ]:
res_f = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df).fit()
res_f.summary()

Как видим, тот же результат получен более удобно, без ручных преобразований признаков. 


С помощью формул также можно катеригоризировать признаки, удалять сонстанту (смещение), конструировать произведение признаков, а также применять к ним функции. Для экономии пространства будем выводить только параметры модели.

In [ ]:
# категоризация
res_f = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region)', data=df).fit()
res_f.params

In [ ]:
# удаляем смещение
res_f = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) - 1', data=df).fit()
res_f.params

In [ ]:
# заменим два признака на их произведение
res_f = smf.ols(formula='Lottery ~ Literacy:Wealth + C(Region) - 1', data=df).fit()
res_f.params

In [ ]:
# добавим произведение двух признаков, но оставим исходные
res_f = smf.ols(formula='Lottery ~ Literacy*Wealth + C(Region) - 1', data=df).fit()
res_f.params

In [ ]:
# применим функцию к признаку
res_f = smf.ols(formula='Lottery ~ np.log(Literacy) + Wealth', data=df).fit()
res_f.params

Но самая полезная киллер-фича формул - возможность использовать их в моделях, не поддреживающих такую запись. Для этого нам понадобится библиотека patsy. 

In [ ]:
import patsy

f = 'Lottery ~ Literacy * Wealth + C(Region) - 1'

y, X = patsy.dmatrices(f, df, return_type='dataframe')

In [ ]:
X.head()

### Робастная регрессия

Иногда в данных присутсвуют выбросы, которые влияют на качество модели. В таких случаях можно использовать линейную модель, устойчивую к ним, - RLM.

In [ ]:
nsample = 50

new_df = pd.DataFrame()
new_df['x1'] = np.linspace(0, 20, nsample)
new_df['x2'] = (new_df['x1'] - 5)**2

sig = 0.3   # маленькая вариация, больше разницы между моделями OLS и RLM
beta = [0.5, -0.0]
kx = np.dot(new_df.values, beta)
y= kx + sig*1. * np.random.normal(size=nsample)
y_true = kx.copy() # значения без выбросов
y[[38,40,42,45,49]] -= 5  # выбросы

In [ ]:
new_df['y'] = y

In [ ]:
ols_res = smf.ols('y ~ x1 + x2', data=new_df).fit()
ols_res.summary()

Обучим RLM модель

In [ ]:
res_rlm = smf.rlm('y ~ x1 + x2', data=new_df).fit()
res_rlm.summary()

In [ ]:
new_df['fitted_ols'] = ols_res.fittedvalues
new_df['fitted_rlm'] = res_rlm.fittedvalues

plt.figure(figsize=(10, 7))
plt.plot(new_df.x1.values, y, 'o', c='b', label='y')
plt.plot(new_df.x1.values, y_true, c='black', label='True')
plt.plot(new_df.x1, new_df.fitted_ols, c='r', label='OLS')
plt.plot(new_df.x1, new_df.fitted_rlm, c='g', label='RLM')
plt.legend(loc="best")

Как видим, RLM справляется с выбросами гораздо лучше OLS. В качестве робастной нормы можно передавать LeastSquares, HuberT, RamsayE, AndrewWave, TrimmedMean, Hampel, and TukeyBiweight из пакета sm.robust.norms (подробнее <a href="https://www.statsmodels.org/stable/rlm.html">тут</a> )

### GLM и компания

Линейная регрессия накладывает на данные некоторые ограничения, в частности, нормальность распределения ошибок и целевой переменной, а также линейность предсказаний. Но что если это не так? Можно долго мучаться с преобразованием данных, а можно посмотреть в сторону GLM - <a href='https://en.wikipedia.org/wiki/Generalized_linear_model'>generalized linear model</a>. GLM часто применяется в страховании для моделирования частоты заявлений об ущербе и среднего размера выплат, чтобы рассчитать размер взноса за полис, а также в других случаях, когда целевая переменная распределена не по нормальному закону.
<br>
<br>
Рассмотрим применение модели.

In [ ]:
ds = sm.datasets.star98.load()
print(sm.datasets.star98.NOTE)

In [ ]:
ds.data[:1]

In [ ]:
ds.endog[:3], ds.exog[:3]

Обучим GLM, указав распределениe sm.families.Binomial

In [ ]:
res = sm.GLM(ds.endog, ds.exog, family=sm.families.Binomial()).fit()
res.summary()

Посмотрим на остатки модели, для этого построим гистограмму распределения дисперсии остатков и построим Q-Q plot с помощью statsmodels.graphics.

In [ ]:
from scipy import stats

plt.hist(res.resid_deviance.copy(), bins=25)
plt.title('Histogram of standardized deviance residuals');

In [ ]:
from statsmodels import graphics

graphics.gofplots.qqplot(resid, line='r')

Видим, что остатки распределены нормально. 

В statsmodels также реализованы расширения GLM: **GEE**, **MixedLM** - для кластеризованных данных, не скореллированных между кластерами, но внутри кластеров, и с наличием случайных корреляций неизвестной природы. Подробнее <a href="https://www.statsmodels.org/stable/gee.html">тут</a> и <a href = "https://www.statsmodels.org/stable/mixed_linear.html">тут</a>

### Логистическая регрессия

Посмотрим, как применить statsmodels к задаче классификации:

In [ ]:
spector_data = sm.datasets.spector.load()
print(sm.datasets.spector.NOTE)

In [ ]:
spector_data.data

In [ ]:
### если при запуске выдается ошибка "module 'scipy.stats' has no attribute 'chisqprob'", 
### раскомментируйте строки ниже
# from scipy import stats
# stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

res_logit = smf.logit('GRADE ~ TUCE + PSI + GPA', data=spector_data.data).fit()
res_logit.summary()

In [ ]:
plt.plot(spector_data.data['PSI'], spector_data.data['GRADE'], 'o', c='b')
plt.plot(spector_data.data['PSI'], res_logit.fittedvalues, 'o', c='g')

### EDA
В модуле statsmodels.grphics есть много других возможностей для визуального анализа данных, посмотрим на некоторые из них.

In [ ]:
ds.exog.shape

In [ ]:
from statsmodels.graphics import boxplots, correlation, regressionplots

boxplots.violinplot(ds.exog[:, :5], positions=[0, 1, 2, 3, 4])

In [ ]:
cm = np.corrcoef(ds.exog[:5])

fig, ax = plt.subplots(figsize=(5, 5))
correlation.plot_corr(cm, ax=ax)

Мощный тул для отображения результатов регрессии:

In [ ]:
f = plt.figure(figsize=(10, 10))
regressionplots.plot_regress_exog(ols_res, 1, fig=f)

### Полезные функции

В библиотеке много полезных функций, сосредоточены они в модуле statsmodels.tools.tools.

Категоризация:

In [ ]:
from statsmodels.tools import tools

tools.categorical(df.Region.values)[:5]

Добавление смещения к признакам, полезно, если исплользуем модель без формул:

In [ ]:
tools.add_constant(df).head()

Метрики точности предсказания:

In [ ]:
from statsmodels.tools import eval_measures

ols_rmse = eval_measures.rmse(y, ols_res.fittedvalues)
ols_mse = eval_measures.mse(y, ols_res.fittedvalues)
print('OLS metrics: {}, {}'.format(ols_rmse, ols_mse))

rlm_rmse = eval_measures.rmse(y_true, res_rlm.fittedvalues)
rlm_mse = eval_measures.mse(y_true, res_rlm.fittedvalues)
print('RLM metrics: {}, {}'.format(rlm_rmse, rlm_mse))

## Заключение

В этом тьюториале мы рассмотрели самые расспространенные приемы моделирования и другие полезные инструменты statsmodels. В библиотеке еще много всего: ANOVA, непараметрические методы, методы прогнозирования временных рядов, статистические тесты.

В рассмотренных случаях библиотека имеет ряд преимуществ перед slkearn:
* лаконичность
* развернутое саммари по модели из коробки
* формулы
* множество статистических интсрументов и проверок
* минималистичность в целом

Так что если вам не нужно что-то сложное типа градиентного бустинга, statsmodels отлично подойдет

Спасибо за внимание!